In [ ]:
from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin
from cuml.model_selection import train_test_split
from cuml.ensemble import VotingRegressor
from cuml.ensemble import GradientBoostingRegressor
from cuml.svm import SVR
from cuml.metrics import mean_absolute_error
from cuml.model_selection import cross_val_score
import mlflow.sklearn
import pandas as pd
import mlflow

In [ ]:
wine = pd.read_csv('winequality-red.csv')

X = wine.drop('quality', axis=1)
y = wine.quality
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# define objective function

def objective(params):
    mlflow.sklearn.autolog()
    with mlflow.start_run(nested=True):
        gbm = GradientBoostingRegressor()
        svr = SVR()
        blender = VotingRegressor(**params, random_state = 0, estimators = [('gbm', gbm), ('svm', svr)], n_jobs = -2)
        
        mae = cross_val_score(blender, X_train, y_train, cv = 5, scoring = "mae")
        return {'loss': min(mae), 'params': params, 'status': STATUS_OK}

In [ ]:
space = {
    "n_estimators": hp.choice("n_estimators", [500, 600, 700]),
    "max_depth": hp.uniform("max_depth", 2, 15),
    "learning_rate": hp.choice("learning_rate", [0.1, 0.01]),
    "C": hp.uniform("C", 5, 15])
}

In [ ]:
mlflow.set_experiment("Hyperopt_Optimization")

with mlflow.start_run():
    best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 10, trials = Trials())
    mlflow.log_params(best)

In [ ]:
trials.results

In [ ]:
best